In [ ]:
"""

Project : Protein Methylation Classification using RBFNN
Course: Pattern Classification
Authors : Ragunath Anbarasu, Yujie Yao, Zixuan Liu 

"""

'\n\nProject : Protein Methylation Classification using RBFNN\nCourse: Pattern Classification\nAuthors : Ragunath Anbarasu, Yujie\xa0Yao, Zixuan\xa0Liu \n\n'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as stats
from sklearn.impute import SimpleImputer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#original_df = pd.read_csv("/content/drive/MyDrive/pattern_classification/csv_result-Descriptors_Training.csv")
#original_df = pd.read_csv("/content/drive/MyDrive/pattern_classification/csv_result-Descriptors_Calibration.csv")
original_df = pd.read_csv("/content/drive/MyDrive/pattern_classification/Blind_Test_features.csv")


In [4]:
original_df.head

<bound method NDFrame.head of       IP_ES_25_N1  Z3_IB_4_N1  Z1_IB_10_N1  ...  Pb_NO_PCR_V  ECI_NO_PCR_CV  class
0            5.11       17.54         6.41  ...         0.01           0.52      P
1           -1.14       33.73         8.27  ...         0.02           0.65      P
2            4.79       32.81         5.82  ...         0.01           0.57      P
3            9.41       15.62        12.71  ...         0.02           0.62      P
4            5.29       14.09         7.46  ...         0.02           0.59      P
...           ...         ...          ...  ...          ...            ...    ...
5145         9.14       17.25        13.25  ...         0.02           0.55      P
5146         5.75       26.32        11.05  ...         0.02           0.61      P
5147         3.79       17.06        10.12  ...         0.02           0.63      P
5148        -1.89       11.41         9.19  ...         0.01           0.62      P
5149        14.41       27.74        12.21  ...         0

In [158]:
print(original_df.shape) 

(4996, 30)


In [142]:
original_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,19988,NaN,NaN,NaN,9994.5,5770.18,1,4997.75,9994.5,14991.2,19988
IP_ES_25_N1,19988,NaN,NaN,NaN,6.05533,3.96034,-4.49,4.77,5.67,6.47,18.69
Z3_IB_4_N1,19988,NaN,NaN,NaN,31.6964,45.6364,6.54,14.81,20.25,29.34,1275.3
Z1_IB_10_N1,19988,NaN,NaN,NaN,18.0177,49.5716,5.13,7.85,9.23,11.45,333.33
Z1_IB_5_N1,19988,NaN,NaN,NaN,19.6449,54.3615,5.03,7.86,9.29,11.47,333.33
Z3_IB_8_N1,19988,NaN,NaN,NaN,30.4707,42.1182,6.42,14.57,19.865,28.85,340.08
ECI_IB_4_N1,19988,NaN,NaN,NaN,83.4732,114.766,14.04,31.4675,45.05,78.27,2333.33
ECI_IB_5_N1,19988,NaN,NaN,NaN,82.9375,114.868,14.52,31.29,44.52,77.56,2333.33
Gs(U)_IB_12_N1,19988,NaN,NaN,NaN,0.0619812,0.0380396,0.02,0.04,0.05,0.07,0.84
Gs(U)_IB_68_N1,19988,NaN,NaN,NaN,0.0624655,0.0379248,0.02,0.04,0.05,0.07,0.84


In [5]:
#df_without_id = original_df.drop(['id'], 1)
df_without_id = original_df
#df_selected_features = original_df.drop(columns=['id', 'Z1_NO_sideL35_M','HP_NO_sideL35_CV','HP_NO_sideL35_CV','Z1_NO_sideR35_CV','Z1_NO_PRT_CV','Z2_NO_AHR_CV','Pa_NO_BSR_SI71','Pb_NO_PCR_V','ECI_NO_PCR_CV'])

In [7]:
#df_selected_features = df_selected_features.replace(['P', 'N'], [1, 0])
df_without_id=df_without_id.replace(['P', 'N'], [1, 0])

In [8]:
def detect_outlier_iterative_IQR(df, cutoff_ratio = 1.07):
    df_copy = df.copy()
    df_prev = pd.DataFrame({'' : []})
    n1 = 1.0
    while True:
        stat = df_copy.describe()
        df_prev = df_copy.copy()
        for column in df.iloc[:, :-1]:
            median, min25, max75 = stat[column]['50%'], stat[column]['25%'], stat[column]['75%']
            iqr = abs(max75-min25)*1.5
            upper_bound = median + iqr
            lower_bound = median - iqr
            df_copy[column] = df_copy[column].mask(df_copy[column] > upper_bound, np.nan)
            df_copy[column] = df_copy[column].mask(df_copy[column] < lower_bound, np.nan)
        n2 = df_copy.isnull().sum().sum()
        #print(df_copy)
        r = n2/n1
        n1 = n2*1.0
        if r < cutoff_ratio:
            break
    return df_prev

In [9]:
def handle_outlier(df):
    df_copy = df.copy() 
    df_copy = df_copy.dropna(thresh=4)
    df_copy.to_csv("df_nan.csv")
    stat = df_copy.describe()
    for column in df_copy.iloc[:, :-1]:
        median = stat[column]['50%']
        df_copy[column] = df_copy[column].fillna(median) 
    return df_copy

In [10]:
 #df_without_outliers = handle_outlier(detect_outlier_iterative_IQR(original_df))
 df_without_outliers = handle_outlier(detect_outlier_iterative_IQR(df_without_id))

In [11]:
df_without_outliers.shape

(5150, 29)

In [120]:
df_without_outliers.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,14403,NaN,NaN,NaN,9933.8,5774.17,1,4946.5,9968,14953.5,19987
IP_ES_25_N1,14403,NaN,NaN,NaN,5.6301,0.631607,4.03,5.31,5.64,5.95,7.29
Z3_IB_4_N1,14403,NaN,NaN,NaN,18.6255,5.78142,6.54,14.3,17.77,22.23,33.3
Z1_IB_10_N1,14403,NaN,NaN,NaN,8.86887,1.56313,5.18,7.75,8.71,9.85,12.7
Z1_IB_5_N1,14403,NaN,NaN,NaN,8.92744,1.57574,5.03,7.78,8.77,9.94,12.76
Z3_IB_8_N1,14403,NaN,NaN,NaN,18.3579,5.63817,6.42,14.26,17.48,21.9,32.6
ECI_IB_4_N1,14403,NaN,NaN,NaN,38.0028,11.8266,14.93,29.53,36.085,44.32,70.57
ECI_IB_5_N1,14403,NaN,NaN,NaN,37.4411,11.3186,14.52,29.52,35.79,43.36,68.97
Gs(U)_IB_12_N1,14403,NaN,NaN,NaN,0.049505,0.012758,0.03,0.04,0.05,0.06,0.08
Gs(U)_IB_68_N1,14403,NaN,NaN,NaN,0.0496619,0.0127642,0.03,0.04,0.05,0.06,0.08


In [125]:
df_without_outliers

,id,IP_ES_25_N1,Z3_IB_4_N1,Z1_IB_10_N1,Z1_IB_5_N1,Z3_IB_8_N1,ECI_IB_4_N1,ECI_IB_5_N1,Gs(U)_IB_12_N1,Gs(U)_IB_68_N1,Gs(U)_IB_58_N1,Gs(U)_IB_60_N1,Z1_NO_sideL35_M,HP_NO_sideL35_CV,Z1_NO_sideR35_CV,Pb_NO_sideR35_S,IP_NO_sideL35_SI71,Z1_NO_PRT_CV,Z2_NO_AHR_CV,Gs(U)_NO_ALR_SI71,Z3_NO_UCR_S,Z3_NO_UCR_N1,ECI_NO_UCR_CV,Pa_NO_BSR_SI71,ISA_NO_NPR_S,Z3_NO_NPR_V,IP_NO_PLR_S,Pb_NO_PCR_V,ECI_NO_PCR_CV,class
0,1,5.59,14.91,8.74,6.45,12.11,28.79,32.40,0.06,0.05,0.04,0.04,0.69,-3.57,6.06,0.94,0.63,10.30,-10.42,0.51,0.69,3.77,0.43,0.53,-0.79,0.68,0.54,0.02,0.59,P
1,2,4.81,16.87,6.58,11.59,17.25,40.40,56.59,0.04,0.06,0.05,0.05,0.79,-4.56,6.69,0.80,0.62,9.22,2.26,0.55,1.32,2.34,0.52,0.49,-1.27,1.39,0.55,0.02,0.62,P
2,3,5.36,13.81,8.25,8.77,14.82,32.42,61.74,0.03,0.06,0.07,0.05,1.07,-5.22,3.33,0.67,0.52,4.69,-2.42,0.54,-0.69,-2.24,0.37,0.47,-0.74,1.08,0.58,0.01,0.56,P
3,4,6.10,17.98,6.41,11.14,17.59,38.26,55.93,0.04,0.05,0.06,0.04,0.79,-3.57,8.46,0.78,0.57,4.55,-2.42,0.45,1.50,2.44,0.40,0.44,-0.28,0.43,0.77,0.02,0.65,P
4,5,5.67,16.41,8.51,7.07,30.95,50.22,39.52,0.05,0.05,0.04,0.03,0.79,-3.57,5.04,1.13,0.54,4.55,-2.42,0.53,1.88,7.72,0.42,0.50,-1.12,0.91,0.71,0.01,0.65,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19983,19984,5.82,17.98,10.36,10.45,17.59,24.51,24.02,0.04,0.06,0.04,0.04,0.79,-1.71,5.59,0.89,0.60,4.22,3.51,0.54,1.40,3.48,0.41,0.57,-1.36,1.08,0.99,0.01,0.55,N
19984,19985,4.62,20.30,9.39,9.30,17.82,29.02,23.37,0.05,0.04,0.04,0.05,0.43,-2.39,7.21,0.59,0.57,3.00,-5.23,0.52,-0.45,-3.72,0.37,0.46,-0.23,1.12,0.75,0.02,0.58,N
19985,19986,6.69,17.98,6.73,8.77,17.59,36.64,36.34,0.04,0.04,0.06,0.05,0.43,-3.57,3.93,0.58,0.56,4.55,-2.61,0.50,0.98,3.77,0.43,0.61,-0.56,0.90,0.93,0.01,0.57,N
19986,19987,5.67,11.67,6.53,6.79,22.80,41.92,36.34,0.07,0.07,0.04,0.06,1.11,-3.57,5.66,0.79,0.55,4.55,-2.77,0.52,-1.31,1.67,0.42,0.56,-0.71,1.25,0.76,0.02,0.57,N


In [126]:
def find_missing(lst):
    return [x for x in range(lst[0], lst[-1]+1) 
                               if x not in lst]
    

In [127]:
def save_removed_outlier(original_df):
    df_nan=pd.read_csv("df_nan.csv")
    df_nan.rename(columns={"Unnamed: 0": "ID"})
    cols_as_np = df_nan[df_nan.columns[0]].to_numpy()
    list_of_indexes=find_missing(cols_as_np)
    removed_rows = original_df.iloc[list_of_indexes, :]
    removed_rows.to_csv("outliers_removed.csv")
      

In [12]:
 df_without_outliers = df_without_outliers.replace(['P', 'N'], [1, 0])

TypeError: ignored

In [167]:
df_without_outliers.shape

(4996, 29)

In [13]:
def standardize(df):
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    x = df.values
    sc = StandardScaler()
    x_scaled = sc.fit_transform(x)
    new_df = pd.DataFrame(x_scaled)
    new_df.columns = df.columns
    if new_df.columns[-1]=='class': #adjust class
        new_df.iloc[:,-1] = new_df.iloc[:,-1].mask(new_df.iloc[:,-1] > sc.mean_[-1], 1)
        new_df.iloc[:,-1] = new_df.iloc[:,-1].mask(new_df.iloc[:,-1] < sc.mean_[-1], 0)
    return new_df

In [14]:
#standardized_df = standardize(df_without_outliers)
standardized_df = standardize(df_without_outliers)

In [128]:
save_removed_outlier(original_df)

In [15]:
standardized_df["class"].replace({1.0: "P", 0.0: "?"}, inplace=True)

In [16]:
standardized_df.head

<bound method NDFrame.head of       IP_ES_25_N1  Z3_IB_4_N1  Z1_IB_10_N1  ...  Pb_NO_PCR_V  ECI_NO_PCR_CV  class
0       -0.628449   -0.180598    -1.676018  ...    -1.187437       0.144786      ?
1        0.005608   -0.141150    -0.364266  ...     0.804517       1.664191      ?
2       -1.135695    2.438355    -2.092112  ...    -1.187437      -1.374619      ?
3        0.005608   -0.509896    -0.089221  ...     0.804517       0.524637      ?
4       -0.343123   -0.772306    -0.935513  ...     0.804517      -0.614916      ?
...           ...         ...          ...  ...          ...            ...    ...
5145     0.005608   -0.230335    -0.089221  ...     0.804517      -2.134321      ?
5146     0.386042    1.325257     1.596311  ...     0.804517       0.144786      ?
5147     0.005608   -0.262922     0.940434  ...     0.804517       0.904488      ?
5148     0.005608   -1.231952     0.284558  ...    -1.187437       0.524637      ?
5149     0.005608    1.568801     2.414393  ...     0.804

In [17]:
#standardized_df.to_csv('Standardized_train.csv',index=False)
#df_without_outliers.to_csv('Standardized_train_no_removed.csv',index=False)
#standardized_df.to_csv('Standardized_val.csv',index=False)
standardized_df.to_csv('Standardized_blind.csv',index=False)